## Set up a duckweed genotypes x media growth assay

### Pre-requisites to use this script
1. Precise plate and well positions for your machine defined in Plate_positions.py in the 'utils' subdirectory. 
2. All the parameters defined in the cell 'User paramters'. 
3. Sufficient 24-well plates, sterile media and duckweed plants available. 
4. Jubilee machine set up with Media-dispensing syringe tool (50 mL), Duckweed transfer syringe tool and lab automation bedplate. 


In [ ]:
#Importing python libraries downloaded from the internet
import random
import pandas as pd
import os
import json
import time

In [ ]:
#Importing python libraries from local files. 
from utils.CameraUtils import *
import utils.DuckbotExptSetupUtils as exp
from utils.MachineUtils import *
import utils.PlatePositionUtils as pp 

In [ ]:
port = "/dev/ttyACM0"
m = MachineCommunication(port)

## 1. Define user parameters

### Labware config 
All should be in arbitrary Jubilee motor units. Find this manually on the machine. They shouldn't need to be updated until you start using a new reservoir. And if that's the case you could create a labware library and then define the media_reservoir variable as one of the objects in your library. 

In [ ]:
#X and Y should get the machine to the center of the reservoir. 
media_reservoir = { 'x' : 75, 'y' : 241,} #You could just point to the middle of position 0. 

#Tool positions
media_syringe = 2 #What jubilee tool position did you define this as?
duckweed_syringe = 3
inoculation_loop = 4

#Volume calculations
dispenses_per_syringe_fill = 20
dispense_mL = 1.5 #In mL
vol_conversion = 3.9 #One mL is 3.9 units. 
dispense_offset = dispense_mL * vol_conversion

#Z-positions for different actions
z_dict = {"zero": 0, #Note that zeros
          "aspirate" : -46.5, #Measure this for yourself and your labware/toolheads.
          "dispense" : -21.5
         }

### Define experimental variables, and desired file names and save locations. 

In [ ]:

#1. DEFINE EXPERIMENTAL VARIABLES
genotypes = ["Sp7498", "Lm5500", "Lm8627"] # Replace with names for unique duckweed genotypes
media = ["Mock"] 

# media = ["Mock", "25 mM Salt", "50 mM Salt", "75 mM Salt","150 mM Salt"] # Replace with names for unique media
reps  = 8 # Replace with your desired number of replicates for each duckweed/media combination. 

#2. DEFINE FILE LOCATION AND NAME
expt_setup_parent_dir = os.getcwd()
# expt_setup_parent_dir = '/Users/Orlando/Documents/Github_clones/duckbot/notebooks/Demo/' # Default uses current working directory but you can replace with your own choice. 
# expt_setup_dir = "TestDriveExpt_1_08032022" # Name of the folder to hold experiment data and metadata including the setup file
# expt_setup_dir = "Test_experiments" 
expt_setup_dir = "SandboxAspiration" 
# expt_setup_filename = "TestDriveExpt_1_08032022.json" #Name for the experiment setup file (Metadata)
# expt_setup_filename = "Expt_082222.json" #Name for the experiment setup file (Metadata)
expt_setup_filename = "SandboxAspiration.json"


expt_setup_file_path = os.path.join(expt_setup_parent_dir, expt_setup_dir)
print(expt_setup_file_path)

if not os.path.exists(expt_setup_file_path):
    os.mkdir(expt_setup_file_path)     

## 2. Define and visualize experimental set up

In [ ]:
# Creates master list of sample info, shuffles and then assigns to plates and wells. 
master_expt_list = []

for g in genotypes:
    for md in media:
        for x in range(reps):
             master_expt_list.append({"genotype": g, "media": md, "condition_replicate": x + 1})


random.shuffle(master_expt_list)
master_expt_list = exp.assign_plates_and_wells(master_expt_list)
expt_dict = {"sample_info" : master_expt_list}

In [ ]:
#Save experimental set up file
os.chdir(expt_setup_file_path)
with open(expt_setup_filename, 'w') as f:
    json.dump(expt_dict, f)

In [ ]:
#Import from file (in case user wants to make any manual edits to the JSON file after creating it)

with open(expt_setup_filename) as datafile:
    expt_data = json.load(datafile)

# Turn samples list into a dataframe
sample_data = expt_data["sample_info"]

In [ ]:
%matplotlib inline
df_with_well_coords = pp.add_well_coords_to_df_from_file(expt_setup_file_path, expt_setup_filename)
exp.visualize_plate_set_up(df_with_well_coords)

## 3. Label 24-well plates and add to machine

#### Label Plates and add plates to machine

In [ ]:
df = pd.DataFrame(sample_data)
num_plates = df.Plate.nunique()
print(num_plates)

print("This experiment requires {} 24-well plate(s)".format(num_plates))
print("----")
lst = list(range(1,num_plates + 1))
for n in lst:
    print("Label a plate with experiment ID or initials and 'plate {}'".format(n))
print("----")
print ("Place the 24-well plate(s) in the jubilee".format(num_plates))
print ("Start at position 1 and fill empty plate slots in order")

## 2. Dispense media
When prompted insert containers of the relevant sterile media into the input slot on the Jubilee. 

In [ ]:
#Retrieve absolute positions of wells from a library and then add those coordinatest to the plate set up dataframe
df = pp.add_well_coords_to_df_from_file(expt_setup_file_path, expt_setup_filename)

In [ ]:
#Reorganizes dataframe to create machine instructions sorted by media-type
media_dicts = pp.pull_list_of_well_coord_dicts_by_dfcolumn(df, 'media')
print(media_dicts)

In [ ]:
#Pick up syringe toolhead
m.toolChange(media_syringe)

In [ ]:
# Send machine instructions
for media in media_dicts:
    m.moveTo(x=0,y=0,z=0)
    print(f"Please ensure {media['media']} is available in the machine before continuing.")
    print("Change syringe and/or needle if desired")
    while True:
        value = input("Enter 'YES' to confirm that the correct media is in position")
        if value != "YES":
            print("Please confirm")
        else:
            break
    exp.dispense_to_wells(m, media["well-coords"], dispense_offset, dispenses_per_syringe_fill, media_reservoir, z_dict)

## 5. Transfer duckweed
Place a container filled with fronds of the relevant duckweed type and the machine will attempt to move individual fronds into the relevant wells. After each attempt at filling all wells a camera will take pictures of each well to confirm success and then unsuccessful wells will be reattempted

In [ ]:
duckweed_reservoir = [75, 241]
#pop the bed down to avoid any collisions on tool change
m.moveTo(z=50)

In [ ]:
# pick up the innoculation loop
m.toolChange(inoculation_loop)
m.moveTo(x=duckweed_reservoir[0], y=duckweed_reservoir[1])

In [ ]:
# find suitable z heights to collect, move, and drop-off duckweed
# this can be done using the duet web control interface
inoculation_loop_z_dict = {"collect": -34.7,  # what height fully immerses the inoculation loop in the reservoir?
                          "move" : -7, # what height clears all labware to move between reservoir/well plates?
                          "transfer" : -32  # what height fully immerses the innoculation loop in the well plate?
                          }

In [ ]:
# pop the bed down to avoid collisions after any probing 
m.moveTo(z=50)

In [ ]:
# Inoculation Loop Transfer
# The machine will move after running this cell
exp.inoculation_loop_transfer(m, df, duckweed_reservoir, inoculation_loop_z_dict)

In [ ]:
# pop bed down to access labware
m.moveTo(z=100)

## Manual Transfer Instructions

In [ ]:
grouped_df = df.groupby('genotype')
for field_value, sample_df in grouped_df:
    print("Place container of duckweed type **{0}** into jubilee and ensure lid is open".format(field_value))
    for index,s in sample_df.iterrows():
        plate = s["Plate"]
        well = s["Well"]
        print(f"Transfer {field_value} to plate {plate}, well {well}")
    input()
        #print("Dispensing media of type {0} into {1}, well {2}".format(field_value,s["Plate"], s["Well"]))


#### Record any extra notes you'd like to add
Write down any notes about today's set up that you would like to be recorded in the set up file 

In [ ]:
os.chdir(expt_setup_file_path)
notes = input()
im.add_input_to_json(notes, "Set_up_notes", expt_setup_filename)

## Consecutive Tranfser Passes
These cells will find any wells missing duckweed, and then prompt you to swap in the appropriate duckweed reservoir for transfer

In [ ]:
m.toolChange(1)
exp.check_wells(m, df)

In [ ]:
exp.fill_empty_wells(m, df, duckweed_reservoir, inoculation_loop_z_dict)